In [1]:
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path  import join
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [2]:
input_path='~/Downloads/'
training_images_path = "/Users/utkarsh/Downloads/MNIST/train-images-idx3-ubyte.gz"
training_labels_path = "/Users/utkarsh/Downloads/MNIST/train-labels-idx1-ubyte.gz"
 
train_images_byte = gzip.open(training_images_path,'r')
image_size = 28
sample_size = 60000 
train_images_byte.read(16)
buf = train_images_byte.read(image_size * image_size * sample_size)
data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)

"""print(np.array2string(data, threshold=np.inf, edgeitems=np.inf))
print(data.reshape(28,28)) 
reshaped_arr = data.reshape(28, 28)
for row in reshaped_arr:
    print(' '.join([str(elem) for elem in row]))"""

images = data.reshape(sample_size, image_size, image_size, 1)
# print(np.array2string(images, threshold=np.inf, edgeitems=np.inf))
train_labels_byte = gzip.open(training_labels_path,'r')
train_labels_byte.read(8)
buf = train_labels_byte.read(sample_size)
labels = np.frombuffer(buf, dtype=np.uint8)

In [3]:
def show_images(images, labels):
     
    cols = 5
    rows = int(len(images)/cols) + 1
     
    plt.figure(figsize=(10,7))
    index = 1   
     
    for x in zip(images, labels):        
        image = x[0]        
        label = x[1]
        plt.subplot(rows, cols, index)        
        plt.imshow(image, cmap=plt.cm.gray, )
        plt.axis('off')
        # plt.title(label);        
        index += 1


In [4]:
"""

Process: (1) Get all the Datasets, their X and Y as numpy array

"""

'\n\nProcess: (1) Get all the Datasets, their X and Y as numpy array\n\n'

In [5]:
# print(np.array2string(data, threshold=np.inf, edgeitems=np.inf))
img_data = np.reshape(data, (sample_size, image_size**2))
lbl_data = labels

In [6]:
X = img_data
y = lbl_data


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [8]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,
    patience=20,
    restore_best_weights=True,
)

In [9]:
model = Sequential(
    [
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        Dense(1000, activation='relu', input_shape=[784]),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        Dense(1000, activation='relu'),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        Dense(1000, activation='relu'),
        layers.BatchNormalization(),
        Dense(10, activation='linear')
    ]
)

In [10]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.legacy.Adam(0.001),
)

In [11]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=5000,
    callbacks=[early_stopping],
)

Epoch 1/5000


 939/1500 [=================>............] - ETA: 12s - loss: 0.5449

KeyboardInterrupt: 

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[5:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));